In [17]:
# pip install openai pandas
import os, json
import pandas as pd
from openai import OpenAI

# --- Config ---
CSV_PATH = "dev_translated.csv"          # adjust path if needed
OUT_PATH = "submission_sample.json"      # output file with 1 sample
MODEL = "gpt-5"                    # pick a fast, capable model

# --- Load one instruction (first row) ---
df = pd.read_csv(CSV_PATH)
row = df.iloc[0]                         # one sample only
sample_id = int(row["id"])
instruction = str(row["instruction_en"]).strip()
API=""
# --- Call OpenAI ---
client = OpenAI(api_key=API)

system_prompt = (
    "You are a precise Python assistant. "
    "Write a correct, concise solution in Python that satisfies the user's instruction. "
    #"Return ONLY in this format:\n"
    "python code here"
)

user_prompt = (
    f"{instruction}\n\n"
    "Constraints:\n"
    "- Return only the code"
    "- Do not include explanations or tests."
)

resp = client.chat.completions.create(
    model=MODEL,
    temperature=1,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

generated = resp.choices[0].message.content

# Safety fallback to guarantee the required prefix exists
generated = generated.lstrip()

# --- Save in submission-like structure (one item) ---
out = [{"id": sample_id, "response": generated}]
with open(OUT_PATH, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print(f"Saved one-sample submission to: {OUT_PATH}")


Saved one-sample submission to: submission_sample.json


In [18]:
# pip install openai pandas
import os, json
import pandas as pd
from openai import OpenAI
from tqdm import tqdm   # optional: shows progress bar

# --- Config ---
CSV_PATH = "dev_translated.csv"     # adjust path if needed
OUT_PATH = "submission.json"        # output file with all rows
MODEL = "gpt-5"

# --- Load CSV ---
df = pd.read_csv(CSV_PATH)

# --- Setup client ---
client = OpenAI(api_key=API)

system_prompt = (
    "You are a precise Python assistant. "
    "Write a correct, concise solution in Python that satisfies the user's instruction. "
    "<python code here>"
)

results = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    sample_id = int(row["id"])
    instruction = str(row["instruction_en"]).strip()

    user_prompt = (
        f"{instruction}\n\n"
        "Constraints:\n"
        "- Return only the code"
        "- Do not include explanations or tests."
    )

    try:
        resp = client.chat.completions.create(
            model=MODEL,
            temperature=1,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        generated = resp.choices[0].message.content
        generated = generated.lstrip()
    except Exception as e:
        # in case of API error, log empty or fallback
        generated = "### Response:\n# ERROR: " + str(e)

    results.append({"id": sample_id, "response": generated})

# --- Save submission ---
with open(OUT_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Saved submission with {len(results)} rows to: {OUT_PATH}")


100%|██████████████████████████████████████████████████████████████████████████████| 400/400 [1:51:25<00:00, 16.71s/it]

Saved submission with 400 rows to: submission.json
